In [1]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
     

import torchvision
import torchvision.transforms as transforms


from models import *
use_gpu = torch.cuda.is_available()

    
    
batch_size = 512




normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

unnec=[]

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))
        if prec<90:
        # compute gradient and do SGD step
         optimizer.zero_grad()
        
         loss.backward()
         optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    adjust_list = [150, 225]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1        

#model = nn.DataParallel(model).cuda()
#all_params = checkpoint['state_dict']
#model.load_state_dict(all_params, strict=False)
#criterion = nn.CrossEntropyLoss().cuda()
#validate(testloader, model, criterion)

Files already downloaded and verified
Files already downloaded and verified


In [2]:


criterion = nn.CrossEntropyLoss().cuda()
model_name='VGG16_quant_project_final'
model =VGG16_quant_project().cuda()

try:
  model.load_state_dict(torch.load('result/'+model_name+'/model_best.pth.tar')['state_dict'])
  best_prec =  validate(testloader, model, criterion)
    
except:
    print("start from scracth")
    best_prec =  0



Test: [0/20]	Time 4.329 (4.329)	Loss 0.2812 (0.2812)	Prec 90.820% (90.820%)
 * Prec 90.160% 


In [3]:

#step 1 create the mask for prunning 1 out of 4 inputs
x=model.features[27].weight
x=x.add(-x.mean())
y=x.reshape(x.shape[0],x.shape[1],-1)
y=y.reshape(-1,y.shape[2]).t()
y=y.reshape(y.shape[0],x.shape[0],x.shape[1])
y=y.reshape(y.shape[0],y.shape[1],-1,2)

mask=torch.ones_like(y)
for i in range(y.shape[0]):
    for j in range(y.shape[1]):
        for k in range(y.shape[2]):
            mask[i][j][k][torch.argmin(y[i][j][k].abs())]=0
global_mask=mask.reshape(9,8,-1).reshape(9,-1).t().reshape(8,8,9).reshape(8,8,3,3).cuda()  

#step 2 prune and change the training algirithm to keep them pruned



In [4]:
model.features[27].weight.data=model.features[27].weight.data*global_mask
model.features[27].weight.data=model.features[27].weight.data+(1-global_mask)*model.features[27].weight.data.mean()*2
best_prec =  validate(testloader, model, criterion)

Test: [0/20]	Time 0.627 (0.627)	Loss 0.2812 (0.2812)	Prec 90.820% (90.820%)
 * Prec 90.160% 


In [5]:
model.features[27].weight_q.reshape(8,8,9).reshape(64,9).t().reshape(9,8,8)[0][3]

tensor([ 0.0000, -1.8412,  0.0000, -0.3069,  0.0000,  1.8412,  1.8412,  0.0000],
       device='cuda:0', grad_fn=<SelectBackward0>)

In [ ]:
save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, True, fdir)

In [ ]:
lr = 1e-5
weight_decay = 1e-5
epochs = 40




optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
# weight decay: for regularization to prevent overfitting


if not os.path.exists('result'):
    os.makedirs('result')
    
fdir = 'result/'+str(model_name)

if not os.path.exists(fdir):
    os.makedirs(fdir)



while best_prec<92:

 for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.5   



 for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch)

    train(trainloader, model, criterion, optimizer, epoch)
    model.features[27].weight.data=model.features[27].weight.data*global_mask
    model.features[27].weight.data=model.features[27].weight.data+(1-global_mask)*model.features[27].weight.data.mean()*2
   
 
        
    # evaluate on test set
    print("Validation starts")
    prec = validate(testloader, model, criterion)

    # remember best precision and save checkpoint
    is_best = prec > best_prec
    best_prec = max(prec,best_prec)
    print('best acc: {:1f}'.format(best_prec))
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, is_best, fdir)




In [ ]:
class SaveOutput:
    def __init__(self):
        self.outputs = []
    def __call__(self, module, module_in):
        self.outputs.append(module_in)
    def clear(self):
        self.outputs = []  
        
######### Save inputs from selected layer ##########
save_output = SaveOutput()
i = 0

for layer in model.modules():
    i = i+1
    if isinstance(layer, QuantConv2d):
        print(i,"-th layer prehooked")
        layer.register_forward_pre_hook(save_output)             
####################################################

for images, labels in testloader:    
    out = model(images.cuda())
    break 


In [ ]:
weight_q = model.features[27].weight_q
w_alpha = model.features[27].weight_quant.wgt_alpha
w_bit = 4

weight_int = weight_q / (w_alpha / (2**(w_bit-1)-1))

In [ ]:
act = save_output.outputs[8][0]
act_alpha  = model.features[27].act_alpha
act_bit = 4
act_quant_fn = act_quantization(act_bit)

act_q = act_quant_fn(act, act_alpha)

act_int = act_q / (act_alpha / (2**act_bit-1))

In [ ]:
conv_int = torch.nn.Conv2d(in_channels = 8, out_channels=8, kernel_size = 3, padding=1)
conv_int.weight = torch.nn.parameter.Parameter(weight_int)
conv_int.bias = model.features[27].bias
output_int = conv_int(act_int)
output_recovered = output_int * (act_alpha / (2**act_bit-1)) * (w_alpha / (2**(w_bit-1)-1))

In [ ]:
output_recovered=nn.ReLU()(output_recovered)

In [ ]:
(save_output.outputs[9][0]-output_recovered).abs().sum()/512

In [ ]:
save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, True, fdir)

In [ ]:
model = VGG16_quant_project().cuda()

In [ ]:
prec =  validate(testloader, model, criterion)

In [ ]:
for i in range(20):
    if(i%4==0):
        pass
    print(i)

In [ ]:
prec=0

In [ ]:
model.features[27].weight_q.shape

In [ ]:
model_name